In [12]:
from lrfhss.run import *
from lrfhss.traffic import *
import time
from joblib import Parallel, delayed
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pickle
import pandas as pd
from matplotlib.ticker import EngFormatter
import scienceplots

# Example 1: Success Probability versus Number of Nodes

Here is a simple example on how to run the simulator using most default input parameter from the class Settings. This is defined in the file runSim, which uses from the simulator classes to create the simulation environment. You can check the file to modified or create you own modified version.

We run the simulation for several different number of devices, and also run it 5 times and get the average and standard deviation of the results. The data is stored in a pandas Dataframe and saved on a .data file.

In [ ]:
#Number of different number of nodes points (each simulation takes one different)
nNodes_points = 20
#Mininum amount of nodes
nNodes_min = 1000
#Maximum amount of nodes
nNodes_max = 100_000
#Number of nodes is divided by 8, as we are simulating one of the 8 grid.
#As they are random selected, it is a very good approximation to consider one of them only, and it decreases the simulation time.
#In the end, we multiply this array by 8 if we want to consider the technology total capacity.
nNodes = np.linspace(nNodes_min, nNodes_max, nNodes_points, dtype=int)//8
#Number of simulation loops for each configuration.
loops = 1000
jobs = 10
hours = 24
df = pd.DataFrame(columns=['Success','Success_ACRDA','Success_std', 'Goodput','Throughput', 'Throughput_std', 'Traffic'])


start = time.perf_counter()

success_exp = []
success_exp_ACRDA = []
success_exp_std = []
goodput_exp = []
throughput_exp = []
throughput_exp_std = []

success_unif = []
success_unif_std = []
goodput_unif = []
throughput_unif = []
throughput_unif_std = []

success_markov = []
success_markov_ACRDA = []
success_markov_std = []
goodput_markov = []
throughput_markov = []
throughput_markov_std = []

#For each number of nodes point, run the simulation "loops" times
for n in nNodes:
    print(f'Nodes: {n*8}')
    ####Exponential Traffic####
    print('Start Exponential')
    #For each nNodes, create a new settings object with the proper input parameter
    s = Settings(simulation_time=60*60*hours, payload_size=20, number_nodes = n, traffic_class=Exponential_Traffic, traffic_param={'average_interval': 900})
    #This line runs the simulation loops in paralel, using n_jobs as the number of threads generated.
    #Consider using a number according to the amount of reseources available to your machine to avoid crashing your system.
    results = Parallel(n_jobs=jobs) (delayed(run_sim)(s, seed = seed) for seed in range(0,loops))

    success_exp.append(np.mean(results,0)[0][0])
    success_exp_std.append(np.std(results,0)[0][0])
    goodput_exp.append(np.mean(results,0)[1][0])
    throughput_exp.append(np.mean(results,0)[2][0])
    throughput_exp_std.append(np.std(results,0)[2][0])

    s = Settings(simulation_time=60*60*hours, payload_size=20, base='acrda', number_nodes=n, traffic_class=Exponential_Traffic, traffic_param={'average_interval': 900})
    results = Parallel(n_jobs=jobs) (delayed(run_sim)(s, seed = seed) for seed in range(0,loops))
    success_exp_ACRDA.append(np.mean(results,0)[0][0])

    ####Uniform Traffic####
    print(f'Start Uniform')

    s = Settings(simulation_time=60*60*hours, payload_size=20, number_nodes=n, traffic_class=Uniform_Traffic, traffic_param={'max_interval': 1800})
    results = Parallel(n_jobs=jobs) (delayed(run_sim)(s, seed = seed) for seed in range(0,loops))

    success_unif.append(np.mean(results,0)[0][0])
    success_unif_std.append(np.std(results,0)[0][0])
    goodput_unif.append(np.mean(results,0)[1][0])
    throughput_unif.append(np.mean(results,0)[2][0])
    throughput_unif_std.append(np.std(results,0)[2][0])

    #Markov parameters
    p = 0.99998
    q = 0.15
    pi_2 = (p-1)/(p-q-1)
    markov_time = pi_2*900
    transition_matrix = [[p, 1-p],[q, 1-q]]

    ####Markov Traffic####
    print(f'Start Markov')

    s = Settings(simulation_time=60*60*hours, payload_size=20, number_nodes=n, traffic_class=Two_State_Markovian_Traffic, traffic_param={'transition_matrix': transition_matrix, 'markov_time': markov_time})
    results = Parallel(n_jobs=jobs) (delayed(run_sim)(s, seed = seed) for seed in range(0,loops))

    success_markov.append(np.mean(results,0)[0][0])
    success_markov_std.append(np.std(results,0)[0][0])
    goodput_markov.append(np.mean(results,0)[1][0])
    throughput_markov.append(np.mean(results,0)[2][0])
    throughput_markov_std.append(np.std(results,0)[2][0])

    s = Settings(simulation_time=60*60*hours, payload_size=20, base='acrda',number_nodes=n, traffic_class=Two_State_Markovian_Traffic, traffic_param={'transition_matrix': transition_matrix, 'markov_time': markov_time})
    results = Parallel(n_jobs=jobs) (delayed(run_sim)(s, seed = seed) for seed in range(0,loops))
    success_markov_ACRDA.append(np.mean(results,0)[0][0])

print(f"The simulation lasted {time.perf_counter()-start} seconds.")

#Saving results on a file
df.loc[len(df.index)] = {'Success': success_exp,'Success_ACRDA': success_exp_ACRDA, 'Success_std': success_exp_std, 'Goodput': goodput_exp,'Throughput': throughput_exp, 'Throughput_std': throughput_exp_std, 'Traffic':'Exponential'}
df.loc[len(df.index)] = {'Success': success_unif,'Success_ACRDA': [], 'Success_std': success_unif_std, 'Goodput': goodput_unif,'Throughput': throughput_unif, 'Throughput_std': throughput_unif_std, 'Traffic':'Uniform'}
df.loc[len(df.index)] = {'Success': success_markov,'Success_ACRDA': success_markov_ACRDA, 'Success_std': success_markov_std, 'Goodput': goodput_markov,'Throughput': throughput_markov, 'Throughput_std': throughput_markov_std, 'Traffic':'Markov'}
file = open('example_traffic_1.data', 'wb')
pickle.dump(df, file)
file.close()

Nodes: 1000
Start Exponential
Start Uniform
Start Markov
Nodes: 6208
Start Exponential
Start Uniform
Start Markov
Nodes: 11416
Start Exponential
Start Uniform
Start Markov
